In [1]:
filenames = [
    'O2 Grouper Pipeline - Task List.xlsx',
    'P2 v3 Authorization_DS manager - list of tasks.xlsx',
    'P2 v3 Replatforming Tasks List.xlsx',
    # 'P2 testing tasks.xlsx',
]


In [2]:
import tools.dev_o2_excel_ingestor as o2

for filename in filenames:
    print("Ingesting: ", filename)
    rdf = o2.generate_rdf_from_xlsx('./tools/', filename)
    o2.send_rdf_to_dgraph('localhost:9080', rdf)
    print("Finished ingestion: ", filename)



Ingesting:  O2 Grouper Pipeline - Task List.xlsx
!!! FINISHED !!!
Finished ingestion:  O2 Grouper Pipeline - Task List.xlsx
Ingesting:  P2 v3 Authorization_DS manager - list of tasks.xlsx
!!! FINISHED !!!
Finished ingestion:  P2 v3 Authorization_DS manager - list of tasks.xlsx
Ingesting:  P2 v3 Replatforming Tasks List.xlsx
!!! FINISHED !!!
Finished ingestion:  P2 v3 Replatforming Tasks List.xlsx


In [1]:
import tools.lib.pmtx_client.query_baselines as rb


url = 'http://localhost:8080/graphql'


query_project_filter = {
    "name": {"allofterms": "P2 v3 Replatforming Tasks List.xlsx"}, 
    "or": {
        "name": {"allofterms": "O2 Grouper Pipeline - Task List.xlsx"}, 
        "or": {
            "name": {"allofterms": "P2 v3 Authorization_DS manager - list of tasks.xlsx"}
        }
    }
}




normalized_baseline = rb.request_and_normalize_baselines_from_pmtx(url, query_project_filter)

dfp, dfd = rb.baseline_to_pandas_df(normalized_baseline)

print("Finished.")


Finished.


In [2]:
from time import time
import pandas as pd

import tools.lib.primitive_estimation as estimator




solver = estimator.ProposeAssigment()

start_date = dfp.start.min()

solver.projects = dfp.copy(deep=True)
solver.dependencies = dfd.copy(deep=True)
# solver.av = pd.DataFrame([], columns=['user_id', 'start', 'finish'])
solver.av = estimator.create_availability_calendar(number_of_users=7)
# solver.av = expander.create_availability_frame()

# Calculation part

solver.initialize(start_date)


algo_time_start = time()


# finish_date = solver.assign_projects_infinite_resources()
# finish_date = solver.assign_projects_to_resources_first_free(one_worker_per_project=True)
# finish_date = solver.assign_projects_by_start_based_on_infinite_resources(one_worker_per_project=True)
finish_date = solver.assign_projects_by_start_based_on_infinite_resources(partial_update=True, partial_update_from=pd.Timestamp('2021-01-01', tz='UTC'), one_worker_per_project=True)

solver.update_projects()

algo_time_finish = time()

print('Project finish timestamp: ' + str(finish_date))
print('Calculation time [s]: ' + str(algo_time_finish - algo_time_start))
print('Unassigned workers time during project: ' + str((solver.av[solver.av.project_id.isnull() & (solver.av.start <= finish_date)].finish - solver.av[solver.av.project_id.isnull() & (solver.av.start <= finish_date)].start).sum()))

print("Finished.")




d:\pmt_x\tools\lib\primitive_estimation.py:289: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.lp.start[self.lp.start.isnull()] = partial_update_from
d:\pmt_x\tools\lib\primitive_estimation.py:145: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_row['project_id'] = None
d:\pmt_x\tools\lib\primitive_estimation.py:147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_row['start'] = self.av.at[index, 'finish']
C:\Users\Art\AppData\

In [4]:
import tools.lib.pmtx_client.mutate_baselines as mb


root_id = "0x743"
# baseline_name = "infinite_resources_6"
baseline_name = "limited_5"


baseline_id = mb.add_baseline_and_return_id(url, baseline_name, root_id)
mb.add_project_baseline(url, solver.projects, baseline_id, root_id)
# TODO: add dependencies

print("Finished.")

Finished.
